In [31]:
import numpy as np
import pyarrow as pa
import h5py
from src.generators import *
np.random.seed(0)

ModuleNotFoundError: No module named 'base'

In [30]:
generator = F1()

NameError: name 'F1' is not defined

In [146]:
with h5py.File('data_hierarchy.h5', 'r') as f:
    image = f["example_0/image_feature/image1/image"]
    print(image)

<HDF5 dataset "image": shape (0,), type "<f8">
